# A/T/G/C-состав, GC, AT, ATG, TGC-состав, содержание K-меров

In [1]:
from Bio import SeqIO

In [2]:
def chrom_intervals_dict(filename):
    chrom_intervals = {}
    with open(filename) as f:
        for line in f:
            name, start, stop = line.split()
            chrom_interval = chrom_intervals.get(name, [])
            chrom_interval.append(tuple([int(start), int(stop)]))
            chrom_intervals[name] = chrom_interval        
    return chrom_intervals
    
def calc_counts(str, chars_names):
    str = str.upper()
    dict ={}
    dict = dict.fromkeys(chars_names, 0)
    for i in xrange(0, len(str)-2):
        nuc_name = str[i] # A, C, T, G
        two_mer_name = nuc_name + str[i+1] # AA, AC, .., GT, GG
        three_mer_name = two_mer_name + str[i+2] # AAA, AAC, ..., GGT, GGG

        dict[nuc_name] = dict.get(nuc_name, 0) + 1;
        dict[two_mer_name] = dict.get(two_mer_name, 0) + 1;
        dict[three_mer_name] = dict.get(three_mer_name, 0) + 1;

    prevlast_nuc_name = str[len(str)-2]
    last_two_mer_name = prevlast_nuc_name + str[len(str)-1]
    last_nuc_name = str[len(str)-1]
    dict[prevlast_nuc_name] = dict.get(prevlast_nuc_name, 0) + 1;
    dict[last_two_mer_name] = dict.get(last_two_mer_name, 0) + 1;
    dict[last_nuc_name] = dict.get(last_nuc_name, 0) + 1;
    # Добавляем АТ и GC:
    dict['A/T'] = dict.get('A', 0) + dict.get('T', 0)
    dict['G/C'] = dict.get('G', 0) + dict.get('C', 0)
    dict['A/T/G'] = dict.get('A', 0) + dict.get('T', 0) + dict.get('G', 0)
    dict['T/G/C'] = dict.get('T', 0) + dict.get('G', 0) + dict.get('C', 0)
    return dict

def get_charachteristics_names():
    names = ['A', 'C', 'T', 'G']
    chars = []
    for n in names:
        chars.append(n)
        for n2 in names:
            chars.append(n + n2)
            for n3 in names:
                chars.append(n + n2 + n3)
    sortedKmers = sorted(chars, key = lambda k: (len(k), k))
    sortedKmers.insert(4, 'G/C')
    sortedKmers.insert(5, 'A/T')
    sortedKmers.insert(6, 'A/T/G')
    sortedKmers.insert(7, 'T/G/C')
    return sortedKmers

#считаем нуклеотидный и динуклеотидный состав (количество)
def calc_new_counts(str, chars_names, dict):
    for i in xrange(0, len(str)-2):
        nuc_name = str[i].upper() # A, C, T, G
        two_mer_name = nuc_name + str[i+1].upper() # AA, AC, .., GT, GG

        dict[nuc_name] = dict.get(nuc_name, 0) + 1;
        dict[two_mer_name] = dict.get(two_mer_name, 0) + 1;

    prevlast_nuc_name = str[len(str)-2].upper()
    last_two_mer_name = prevlast_nuc_name + str[len(str)-1].upper()
    last_nuc_name = str[len(str)-1].upper()
    dict[prevlast_nuc_name] = dict.get(prevlast_nuc_name, 0) + 1;
    dict[last_two_mer_name] = dict.get(last_two_mer_name, 0) + 1;
    dict[last_nuc_name] = dict.get(last_nuc_name, 0) + 1;
    return dict

In [3]:
len(get_charachteristics_names())

88

In [66]:
chroms_intervals = chrom_intervals_dict('../data/genome.windows.bed')

In [88]:
def get_chrom_name(fasta):
    # Достаем имя хромосомы и последовательность
    chrom_name_def = "Chromosome: "
    chrom = fasta.description
    index = chrom.find(chrom_name_def)
    right_bound_index = None
    #print index, right_bound_index
    if (index == -1):
        chrom_name_def = 'chr'
        index = chrom.index(chrom_name_def)
    else:
        right_bound_index = index + chrom[index:].index(" ")
        index = chrom.index(chrom_name_def) + len(chrom_name_def)
    name = chrom[index : right_bound_index]
    #print chrom, index, right_bound_index, name
    return name

In [246]:
fasta_sequences = SeqIO.parse(open('../data/dicty.fa'),'fasta')
with open('../data/my_res.tsv', 'w') as out, open('../data/my_res2.tsv', 'w') as out2:
    # Записываем названия колонок для итогового файла
    charachteristics_names = get_charachteristics_names()
    columns = str(charachteristics_names).replace('[', '')
    columns = columns.replace(']', '')
    columns = columns.replace('\'', '')
    columns = columns.replace(', ', '\t')
    out.write('chrom\tstart of bin\tend of bin\t' + columns + '\n')
    out2.write('nucleotides\tfrequency\tbins\t\n')
    
    for fasta in fasta_sequences:
        # Достаем имя хромосомы и последовательность
        chrom = fasta.description
        name = fasta.name
        sequence = fasta.seq.tostring()
        
        for chrom_interval in chroms_intervals[name]:
            start = chrom_interval[0]
            stop = chrom_interval[1]
            charachteristics = calc_counts(sequence[start : stop], charachteristics_names) # Считаем количество 
            length = (stop-start) - sequence[start : stop].count('n') - sequence[start : stop].count('N') + 1
            charachteristics = {k: round((v * 1.0) / (length-len(k)), 2) for k, v in charachteristics.iteritems()} # Считаем в %
            
            for (key, value) in charachteristics.iteritems():
                row2 = key + '\t' + str(value) + '\t' + str(start) + '_' + str(stop) + '\t\n'
                out2.write(row2)
            
            row = name + '\t' + str(start) + '\t' + str(stop) + '\t'
            for char_name in charachteristics_names:
                row = row + str(charachteristics.get(char_name, 0)) + '\t'
            out.write(row + '\n')
            

## Новые данные

In [213]:
# с новым файлом
#genome_names = ['danRer7', 'danRer10', 'galGal4', 'hg19', 'hg38', 'dm3', 'dm6', 'ce10', 'dicty']
genome_names = ['dm3', 'ce10']
file_path = '/Users/victoria/Desktop/ДИПЛОМ/MotifsSearch/data/genomes/'
all_genomes_freqs = {}
for genome_name in genome_names:
    filename = file_path + genome_name + '/' + genome_name + '.fa'
    print filename
    fasta_sequences = SeqIO.parse(open(filename),'fasta')
    with open('../data/new_res_' + genome_name + '.tsv', 'w') as out:
        # Записываем названия колонок для итогового файла
        charachteristics_names = get_charachteristics_names()
        columns = str(charachteristics_names).replace('[', '')
        columns = columns.replace(']', '')
        columns = columns.replace('\'', '')
        columns = columns.replace(',', '\t')
        out.write('chrom\t' + columns + '\n')
        charachteristics ={}
        charachteristics = dict.fromkeys(charachteristics_names, 0)
        genome_len = 0

        for fasta in fasta_sequences:
            # Достаем имя хромосомы и последовательность
            name = fasta.name #get_chrom_name(fasta)
            sequence = fasta.seq.tostring()

            chrom_len = len(sequence) - sequence.count('N') - sequence.count('n')
            genome_len = genome_len + chrom_len
            charachteristics = calc_new_counts(sequence, charachteristics_names, charachteristics) # Считаем количество 

        charachteristics_percent = {k: round((v * 1.0) / (genome_len-len(k)+1), 2) for k, v in charachteristics.iteritems()} # Считаем в %
        all_genomes_freqs[genome_name] = charachteristics_percent # запоминаем состав для всех геномов в словаре
        row = 'croms' + '\t'
        percent_row = 'percents' + '_%\t'
        for char_name in charachteristics_names:
            row = row + str(charachteristics.get(char_name, 0)) + '\t'
            percent_row = percent_row + str(charachteristics_percent.get(char_name, 0)) + '\t'
        out.write(row + '\n')
        out.write(percent_row + '\n')

/Users/victoria/Desktop/ДИПЛОМ/MotifsSearch/data/genomes/dm3/dm3.fa
/Users/victoria/Desktop/ДИПЛОМ/MotifsSearch/data/genomes/ce10/ce10.fa
